In [1]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

plt.ion()

In [2]:
# goal: use a RL agent to approximate y = sin x

In [3]:
# ground truth values

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)
x = x.unsqueeze(-1).pow(torch.tensor([1, 2, 3]))

In [4]:
n_states = 3
n_actions = 2 

In [5]:
class Model(nn.Module):
    def __init__(self, n_states, n_actions):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(n_states, 10)
        self.layer2 = nn.Linear(10, n_actions)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        return self.layer2(x)

Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state'))

class Memory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
    def push(self, *args):
        self.memory.append(Transition(*args))
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    def __len__(self):
        return len(self.memory)
    
class Agent:
    def __init__(self, n_states, n_actions):
        self.batch_size = 128
        self.gamma = 0.7
        self.epsilon = 0.99
        self.decay = 0.01
        self.lr = 1e-4
        
        self.model = Model(n_states, n_actions)
        self.memory = Memory(1000)
        self.optimizer = optim.AdamW(self.model.parameters(), lr=self.lr, amsgrad=True)
    
    def get_action(self, state):
        sample = random.random()
        epsilon = self.epsilon*self.decay
        if sample > epsilon:
            with torch.no_grad():
                return self.model(state).max(1).indices.view(1, 1)
        else:
            random_action = random.choice([0, 1])
            return torch.tensor([[random_action]], dtype=torch.long)
        
    def update(self):
        if len(self.memory) < self.batch_size:
            return
        transitions = self.memory.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        next_state_batch = torch.cat(batch.next_state)
        
        q_values = self.model(state_batch).gather(1, action_batch)
        with torch.no_grad():
            next_state_values = self.model(next_state).max(1).values
        q_pred = (next_state_values * self.gamma) + reward_batch
        
        criterion = nn.MSELoss() # use Huber loss here to smooth results
        loss = criterion(q_values, q_pred.unsqueeze(1)) 
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [6]:
# get step functon that returns next_state and reward
# increase or decrease y-value of y = sin x plot by 0.01
# goal is to have this be user-defined

def step(state, action):
    if action == 1:
        next_state = state + 0.01
    else:
        next_state = state - 0.01
    reward = torch.tensor([1], dtype=torch.long)
    return next_state, reward

In [7]:
agent = Agent(n_states, n_actions)

episodes = []
state = torch.tensor([[-3.1, 9.8, -30.0]])

for i in range(10):
    action = agent.get_action(state)
    print(state, action)
    next_state, reward = step(state, action)
    agent.memory.push(state, action, reward, next_state)
    state = next_state
    agent.update()

tensor([[ -3.1000,   9.8000, -30.0000]]) tensor([[0]])
tensor([[ -3.1100,   9.7900, -30.0100]]) tensor([[0]])
tensor([[ -3.1200,   9.7800, -30.0200]]) tensor([[0]])
tensor([[ -3.1300,   9.7700, -30.0300]]) tensor([[0]])
tensor([[ -3.1400,   9.7600, -30.0400]]) tensor([[0]])
tensor([[ -3.1500,   9.7500, -30.0500]]) tensor([[0]])
tensor([[ -3.1600,   9.7400, -30.0600]]) tensor([[0]])
tensor([[ -3.1700,   9.7300, -30.0700]]) tensor([[0]])
tensor([[ -3.1800,   9.7200, -30.0800]]) tensor([[0]])
tensor([[ -3.1900,   9.7100, -30.0900]]) tensor([[0]])


In [8]:
agent.memory.sample(10)

[Transition(state=tensor([[ -3.1200,   9.7800, -30.0200]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[ -3.1300,   9.7700, -30.0300]])),
 Transition(state=tensor([[ -3.1700,   9.7300, -30.0700]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[ -3.1800,   9.7200, -30.0800]])),
 Transition(state=tensor([[ -3.1400,   9.7600, -30.0400]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[ -3.1500,   9.7500, -30.0500]])),
 Transition(state=tensor([[ -3.1900,   9.7100, -30.0900]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[ -3.2000,   9.7000, -30.1000]])),
 Transition(state=tensor([[ -3.1300,   9.7700, -30.0300]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[ -3.1400,   9.7600, -30.0400]])),
 Transition(state=tensor([[ -3.1000,   9.8000, -30.0000]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[ -3.1100,   9.7900, -30.0100]])),
 Transition(state=tensor([[ -3.1800,   9.7200, -30.0800]]), acti

In [9]:
agent.model.state_dict()

OrderedDict([('layer1.weight',
              tensor([[-0.0297,  0.4871, -0.0171],
                      [-0.4319,  0.0688,  0.2384],
                      [ 0.1314,  0.2184,  0.0897],
                      [-0.2597, -0.3488,  0.2309],
                      [ 0.0962, -0.3886,  0.0303],
                      [-0.3578,  0.2764,  0.5335],
                      [ 0.2767, -0.5136,  0.1847],
                      [ 0.2969,  0.5040,  0.3995],
                      [-0.2637, -0.1289,  0.0143],
                      [ 0.1578, -0.5228, -0.1580]])),
             ('layer1.bias',
              tensor([-0.0792, -0.5499,  0.4758, -0.5061,  0.5100, -0.3410,  0.0103, -0.0888,
                       0.1776, -0.5216])),
             ('layer2.weight',
              tensor([[-0.0397,  0.0260, -0.0692, -0.2167,  0.0625, -0.1226, -0.1632,  0.2986,
                       -0.2895,  0.2575],
                      [-0.1252,  0.2303,  0.0686,  0.0057, -0.2941,  0.2286,  0.1180,  0.0467,
                        0.2

In [10]:
# plotting state_dict function should get an approximation of sin x

In [15]:
linear_layer = agent.model.layer2

In [16]:
linear_layer.weight[:, 0]

tensor([-0.0397, -0.1252], grad_fn=<SelectBackward0>)

In [19]:
y_result = linear_layer.bias[0].item() + \
           linear_layer.weight[:, 0][0].item()*x + \
           linear_layer.weight[:, 1][0].item()*x**2 + \
           linear_layer.weight[:, 2][0].item()*x**3

In [22]:
y_result

tensor([[ 2.7396e+00, -6.4166e+01,  2.0891e+03],
        [ 2.7325e+00, -6.3777e+01,  2.0704e+03],
        [ 2.7255e+00, -6.3390e+01,  2.0519e+03],
        ...,
        [-1.7883e+00, -6.3390e+01, -2.0021e+03],
        [-1.7944e+00, -6.3777e+01, -2.0203e+03],
        [-1.8004e+00, -6.4166e+01, -2.0386e+03]])